# DiploDatos 2019 - Análisis de Series Temporales

## Integrante

| Nombre | e-mail |
|------|------|
|Rivadero, Isabel | isarivadero@hotmail.com |

## Práctico de Aprendizaje supervisado y no supervisado
Continuo trabajando sobre aprendizaje automático: sobre modelos supervisados y no supervisados. Diseño e implemento algunos modelos y defino métricas para ver como performan.

In [1]:
### Aumentar el ancho del notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler

plt.rcParams['figure.figsize'] = (12,8)

holidays = './holidays.csv'
cols = ['service',
        'sender_zipcode',
        'receiver_zipcode',
        'sender_state',
        'receiver_state',
        'shipment_type',
        'quantity',
        'status',
        'date_created',
        'date_sent',
        'date_visit',
        'target']
data_path = './shipments_BR_201903.csv'


In [3]:
def ontimesv(y_test, y_pred):
    ontime_sv= (y_pred == y_test)
    return np.sum(ontime_sv) / np.size(y_test)

def delaysv(y_test, y_pred):
    delay_sv = (y_pred < y_test)
    return np.sum(delay_sv) / np.size(y_test)

def earlysv(y_test, y_pred):
    early_sv= (y_test < y_pred)
    return np.sum(early_sv) / np.size(y_test)

def offset_window(y_test, lower_bound, upper_bound, length):
    offset_msk = ((upper_bound - lower_bound) == length)
    return np.sum(offset_msk) / np.size(offset_msk)


def avg_speed(y_test, lower_bound, upper_bound):
    return lower_bound.mean()


def avg_offset(y_test, lower_bound, upper_bound):
    return (upper_bound - lower_bound).mean()

def get_metrics(y_test, speed, offset):
    lower_bound = speed
    upper_bound = speed + offset
    metrics = {'on_time': ontime(y_test, lower_bound, upper_bound).astype(float).round(3),
               'delay': delay(y_test, lower_bound, upper_bound).astype(float).round(3),
               'early': early(y_test, lower_bound, upper_bound).astype(float).round(3),
               'offset_0': offset_window(y_test, lower_bound, upper_bound, 0).astype(float).round(3),
               'offset_1': offset_window(y_test, lower_bound, upper_bound, 1).astype(float).round(3),
               'offset_2': offset_window(y_test, lower_bound, upper_bound, 2).astype(float).round(3),
               'avg_speed': avg_speed(y_test, lower_bound, upper_bound).astype(float).round(3),
               'avg_offset': avg_offset(y_test, lower_bound, upper_bound).astype(float).round(3),
               }

    return metrics

#### Referencia de las columnas
* **service**: Identificador unico que corresponde a un tipo de servicio de un correo en particular.
* **sender_zipcode:** Código postal de quien envía el paquete (usualmente el vendedor).
* **receiver_zipcode:** Código postal de quien recibe el paquete (usualmente el comprador).
* **sender_state:** Nombre abreviado del estado de quien envía el paquete.
* **receiver_state:** Nombre abreviado del estado de quien recibe el paquete.
* **quantity:** Cantidad de items que tiene dentro el paquete.
* **status:** Estado final del envío.
* **date_created:** Fecha de compra de el o los items.
* **date_sent:** Fecha en que el correo recibe el paquete.
* **date_visit:** Fecha en que el correo entrega el paquete.
* **target:** Cantidad de dias hábiles que tardó el correo en entregar el paquete desde que lo recibe.

In [4]:
df = pd.read_csv(data_path, usecols=cols)
df.shape

(1000000, 12)

In [5]:
df.head()

,sender_state,sender_zipcode,receiver_state,receiver_zipcode,shipment_type,quantity,service,status,date_created,date_sent,date_visit,target
0,SP,3005,SP,5409,express,1,0,done,2019-03-04 00:00:00,2019-03-05 13:24:00,2019-03-07 18:01:00,2
1,SP,17052,MG,37750,standard,1,1,done,2019-03-19 00:00:00,2019-03-20 14:44:00,2019-03-27 10:21:00,5
2,SP,2033,SP,11040,express,1,0,done,2019-02-18 00:00:00,2019-02-21 15:08:00,2019-02-28 18:19:00,5
3,SP,13900,SP,18500,express,1,0,done,2019-03-09 00:00:00,2019-03-11 15:48:00,2019-03-12 13:33:00,1
4,SP,4361,RS,96810,express,1,0,done,2019-03-08 00:00:00,2019-03-12 08:19:00,2019-03-16 08:24:00,4


In [6]:
# set seed for reproducibility
np.random.seed(0)

In [7]:
df.dtypes

sender_state        object
sender_zipcode       int64
receiver_state      object
receiver_zipcode     int64
shipment_type       object
quantity             int64
service              int64
status              object
date_created        object
date_sent           object
date_visit          object
target               int64
dtype: object

Como las fechas estan como tipo objeto no voy a poder operar entonces paso a tipo fecha

In [8]:
df1= df.copy()

In [9]:
df1['date_created']= df1.date_created.astype('datetime64')

In [10]:
df1['date_sent']= df1.date_sent.astype('datetime64')

In [11]:
df1['date_visit']= df1.date_visit.astype('datetime64')

**Eliminamos datos inconsistentes:**

Aplicamos curacion y limpieza de datos

In [12]:
df_clean = df1[(df1['date_sent'] <= df1['date_visit']) & (df1['date_created'] <= df1['date_sent']) & (df1['date_created'] <= df1['date_visit'])]
df1.shape

(1000000, 12)

## Preparación de los features
#### Diseñar un pipeline con las siguientes transformaciones:
1- Recortar el último dígito de los zip codes


In [13]:
df1['sender_zipcode']=df1['sender_zipcode']/10
df1['sender_zipcode'] =df1['sender_zipcode'].astype('int32')
df1['receiver_zipcode']=df1['receiver_zipcode']/10
df1['receiver_zipcode'] =df1['receiver_zipcode'].astype('int32')

In [14]:
df1.dtypes

sender_state                object
sender_zipcode               int32
receiver_state              object
receiver_zipcode             int32
shipment_type               object
quantity                     int64
service                      int64
status                      object
date_created        datetime64[ns]
date_sent           datetime64[ns]
date_visit          datetime64[ns]
target                       int64
dtype: object

In [15]:
df1.head()

,sender_state,sender_zipcode,receiver_state,receiver_zipcode,shipment_type,quantity,service,status,date_created,date_sent,date_visit,target
0,SP,300,SP,540,express,1,0,done,2019-03-04,2019-03-05 13:24:00,2019-03-07 18:01:00,2
1,SP,1705,MG,3775,standard,1,1,done,2019-03-19,2019-03-20 14:44:00,2019-03-27 10:21:00,5
2,SP,203,SP,1104,express,1,0,done,2019-02-18,2019-02-21 15:08:00,2019-02-28 18:19:00,5
3,SP,1390,SP,1850,express,1,0,done,2019-03-09,2019-03-11 15:48:00,2019-03-12 13:33:00,1
4,SP,436,RS,9681,express,1,0,done,2019-03-08,2019-03-12 08:19:00,2019-03-16 08:24:00,4


2- Normalizar los features para que queden en el rango (0, 1)

In [16]:
features = ['sender_zipcode', 'receiver_zipcode', 'service']
target = 'target'

In [17]:
cut_off = '2019-03-20'
df_train = df.query(f'date_visit <= "{cut_off}"')
df_test = df.query(f'date_created > "{cut_off}"')

X_train = df_train[features].values.astype(np.float)
y_train = df_train[target].values

X_test = df_test[features].values.astype(np.float)
y_test = df_test[target].values

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((673645, 3), (673645,), (76378, 3), (76378,))

## preguntar porque lo transforma a numpy con np.float o values, que cambia si no ponemos eso?

In [20]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_n=scaler.transform(X_train)
scaler.fit(X_test)
X_test_n=scaler.transform(X_test)

In [23]:
X_train_n

array([[0.02024876, 0.0445302 , 0.        ],
       [0.01042751, 0.10141531, 0.        ],
       [0.13033374, 0.17677722, 0.        ],
       ...,
       [0.06156473, 0.60431967, 0.09090909],
       [0.03493013, 0.37791068, 0.09090909],
       [0.1650921 , 0.02473002, 0.09090909]])

In [24]:
X_test_n

array([[0.88959969, 0.86685054, 0.1       ],
       [0.35835632, 0.88254074, 0.1       ],
       [0.06140537, 0.19184878, 0.        ],
       ...,
       [0.07639289, 0.11360996, 0.        ],
       [0.12331605, 0.06131604, 0.        ],
       [0.3880686 , 0.03471443, 0.        ]])

3- Proyectar los features utilizando PCA, manteniendo 3 componentes.
#### NOTA IMPORTANTE: 
Estas transformaciones se deben aplicar sin modificar el dataframe con los datos originales, pueden usar copias para hacer las pruebas. Es decir que no deben hacer las transformaciones y guardarlas en un dataframe, tal como se hace en el ejemplo.

## PCA preguntarrr

In [18]:
from sklearn.decomposition import PCA

In [22]:
pca = PCA(n_components=3)
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [25]:
pca.n_components_ 

3

In [26]:
pca.transform(X_train)

array([[-4.08603379e+04, -4.85691094e+03, -1.22822702e+00],
       [-3.59869638e+04, -7.84072901e+03, -1.24328056e+00],
       [-2.46701287e+04,  4.30095151e+02, -1.17465126e+00],
       ...,
       [ 1.21421767e+04, -2.15318603e+04, -2.84429183e-01],
       [-9.65780269e+03, -1.57002964e+04, -2.68311202e-01],
       [-3.73849880e+04,  9.18786630e+03, -1.28745915e-01]])

## Preparación del target:
4- Limitar el target a 20, es decir asignar todo target mayor que 20 a 20.

In [29]:
df1.loc[df1['target']>20, 'target'] = 20

In [30]:
df1.target.value_counts()

1     159537
2     135145
3     107500
4      82250
5      68790
6      59190
7      51902
8      47937
9      42376
10     40461
11     35821
12     30215
13     25068
20     21344
0      21118
14     19853
15     16241
16     12133
17      9724
18      7340
19      6055
Name: target, dtype: int64

## Preparación del dataset:
5- Particionar el dataset en train y test, teniendo los cuidados necesarios para no romper la temporalidad de los datos. El conjunto de training no puede tener menos del 50% de los datos.

#### ya esta hecho en 2

6- Si les parece necesario, pueden realizar algún tipo de filtrado o limpieza de los
datos, explicando por qué les parece necesario.


## ya lo hicimos antes, pero vimos que no habia datos incosistentes porque ninguno quedo filtrado entonces no seria necesario

## Modelo basado en árboles de decisión (supervisado)
7- Crear un pipeline con los pasos de “preparación de los features” agregando el clasificador XGBoostClassifier como estimador final. Entrenar este modelo, predecir el conjunto de test y calcular las métricas ontime, delay y early, sin ventana (se puede utilizar un array con ceros como en el ejemplo).

In [31]:
from xgboost import XGBClassifier

In [33]:
model = Pipeline([
    ('normalizer', MinMaxScaler()),
    ('classifier', XGBClassifier()),
])

In [34]:
%%time
model.fit(X_train, y_train)

CPU times: user 31min 31s, sys: 1.14 s, total: 31min 32s
Wall time: 31min 55s


Pipeline(memory=None,
     steps=[('normalizer', MinMaxScaler(copy=True, feature_range=(0, 1))), ('classifier', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,...
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1))])

In [35]:
y_pred = model.predict(X_test)

In [37]:
y_pred

array([6, 5, 2, ..., 1, 1, 2])

In [42]:
metrics = {
    'ontimesv': ontimesv(y_test, y_pred),
    'delaysv': delaysv(y_test, y_pred),
    'earlysv': earlysv(y_test, y_pred),
}

metrics

{'ontimesv': 0.5011652570111812,
 'delaysv': 0.2351200607504779,
 'earlysv': 0.2637146822383409}

8- Explicar muy brevemente como funcionan esta clase de modelos.

## leer documentacion de xgboost

## Modelo basado en vecinos cercanos (no supervisado / semi supervisado)
9- Crear un pipeline con los pasos de “preparación de los features” agregando el clasificador KNeighborsClassifier como estimador final. Entrenar este modelo, predecir el conjunto de test y calcular las métricas ontime, delay y early, sin ventana.

In [43]:
from sklearn.neighbors import KNeighborsClassifier

In [49]:
model = Pipeline([
    ('normalizer', MinMaxScaler()),
    ('classifier',  KNeighborsClassifier(n_neighbors=3)),
])

In [50]:
%%time
model.fit(X_train, y_train)

CPU times: user 3min 8s, sys: 676 ms, total: 3min 9s
Wall time: 3min 8s


Pipeline(memory=None,
     steps=[('normalizer', MinMaxScaler(copy=True, feature_range=(0, 1))), ('classifier', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform'))])

In [51]:
y_pred = model.predict(X_test)

In [52]:
y_pred

array([4, 7, 1, ..., 1, 1, 2])

In [53]:
metrics = {
    'ontimesv': ontimesv(y_test, y_pred),
    'delaysv': delaysv(y_test, y_pred),
    'earlysv': earlysv(y_test, y_pred),
}

metrics

{'ontimesv': 0.43270313441043234,
 'delaysv': 0.2867710597292414,
 'earlysv': 0.28052580586032627}

10- Explicar muy brevemente como funcionan esta clase de modelos.

## ver documentacion de KNeighborsClassifier

Modelo basado en regresión:
11- Crear un pipeline con los pasos de “preparación de los features” agregando un regresor a elección de ustedes como estimador final. Este regresor puede ser tanto supervisado como no supervisado.

In [18]:
from sklearn.linear_model import LogisticRegression

In [20]:
model = Pipeline([
    ('normalizer', MinMaxScaler()),
    ('classifier', LogisticRegression(solver ='newton-cg',multi_class ='multinomial')),
])

12- Entrenar este modelo, predecir el conjunto de test y redondear las predicciones de forma inteligente. Explicar el criterio de redondeo.

In [21]:
%%time
model.fit(X_train, y_train)

CPU times: user 48min 23s, sys: 16min 25s, total: 1h 4min 48s
Wall time: 54min 7s


Pipeline(memory=None,
     steps=[('normalizer', MinMaxScaler(copy=True, feature_range=(0, 1))), ('classifier', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False))])

In [22]:
y_pred = model.predict(X_test)

In [23]:
y_pred

array([8, 3, 1, ..., 1, 1, 1])

## como que redondear?

13- Calcular las métricas ontime , delay y early, sin ventana.

In [24]:
metrics = {
    'ontimesv': ontimesv(y_test, y_pred),
    'delaysv': delaysv(y_test, y_pred),
    'earlysv': earlysv(y_test, y_pred),
}

metrics

{'ontimesv': 0.4323234439236429,
 'delaysv': 0.40668778967765584,
 'earlysv': 0.1609887663987012}

14- Justificar muy brevemente la elección del modelo.

Ventanas de predicción:
15- Construir un offset para mejorar las predicciones de nuestros modelos, de forma que tenga avg_offset menor o igual a 1, recalcular las métricas y explicar cómo se lo construyó.

In [25]:
def ontime(y_test, lower_bound, upper_bound):
    ontime_msk = (lower_bound <= y_test) & (y_test <= upper_bound)
    return np.sum(ontime_msk) / np.size(y_test)

def delay(y_test, lower_bound, upper_bound):
    delay_msk = (upper_bound < y_test)
    return np.sum(delay_msk) / np.size(y_test)

def early(y_test, lower_bound, upper_bound):
    early_msk = (y_test < lower_bound)
    return np.sum(early_msk) / np.size(y_test)

def offset_window(y_test, lower_bound, upper_bound, length):
    offset_msk = ((upper_bound - lower_bound) == length)
    return np.sum(offset_msk) / np.size(offset_msk)

def avg_speed(y_test, lower_bound, upper_bound):
    return lower_bound.mean()

def avg_offset(y_test, lower_bound, upper_bound):
    return (upper_bound - lower_bound).mean()

def get_metrics(y_test, speed, offset):
    lower_bound = speed
    upper_bound = speed + offset
    metrics = {'on_time': ontime(y_test, lower_bound, upper_bound).astype(float).round(3),
               'delay': delay(y_test, lower_bound, upper_bound).astype(float).round(3),
               'early': early(y_test, lower_bound, upper_bound).astype(float).round(3),
               'offset_0': offset_window(y_test, lower_bound, upper_bound, 0).astype(float).round(3),
               'offset_1': offset_window(y_test, lower_bound, upper_bound, 1).astype(float).round(3),
               'offset_2': offset_window(y_test, lower_bound, upper_bound, 2).astype(float).round(3),
               'avg_speed': avg_speed(y_test, lower_bound, upper_bound).astype(float).round(3),
               'avg_offset': avg_offset(y_test, lower_bound, upper_bound).astype(float).round(3),
               }

    return metrics

In [26]:
y_pred = model.predict(X_test)
offset = np.zeros_like(y_pred)
get_metrics(y_test, y_pred, offset)

{'on_time': 0.432,
 'delay': 0.407,
 'early': 0.161,
 'offset_0': 1.0,
 'offset_1': 0.0,
 'offset_2': 0.0,
 'avg_speed': 1.691,
 'avg_offset': 0.0}

16- Construir un offset que mejore las métricas de los modelo y que además tenga un avg_offset menor o igual que 2.5, recalcular las métricas y explicar cómo se lo construyó.

In [ ]:
y_pred = model.predict(X_test)
offset = np.zeros_like(y_pred)
get_metrics(y_test, y_pred, offset)

## Como generar el offset, como generar numeros aleatorios?